In [1]:
import os
import rasterio as rio
from rasterio.plot import show
from pathlib import Path

In [2]:
dim = 224

In [3]:
os.makedirs('./data/hed/x', exist_ok=True)
os.makedirs('./data/hed/y', exist_ok=True)

In [4]:
img = rio.open('./data/six_bands/2017.10.15.tif')
d = img.read()
show(d[2, 0:224, 0:224], cmap='tab20')

<Figure size 640x480 with 1 Axes>

In [5]:
def generate(in_dir, filename, out_dir, dim):
    y_img = rio.open(Path('./data/truth.tif'))  
    y_data = y_img.read()
    
    y_profile = y_img.profile
    y_profile['width'] = dim
    y_profile['height'] = dim
 
    x_img = rio.open(Path(in_dir) / filename)
    x_data = x_img.read()
    
    x_profile = x_img.profile
    x_profile['width'] = dim
    x_profile['height'] = dim
    
    count = 0
    
    for i in range(0, x_data.shape[1]-dim, dim):
        for j in range(0, x_data.shape[2]-dim, dim):
            name = filename[0:filename.rfind('.')] + '_' + str(count) + '.tif'
            x_filename = Path(out_dir) / 'x' / name
            y_filename = Path(out_dir) / 'y' / name
            
            with rio.open(x_filename, 'w', **x_profile) as x_file:
                x_file.write(x_data[:, i:i+dim, j:j+dim])
                
            with rio.open(y_filename, 'w', **y_profile) as y_file:
                y_file.write(y_data[:, i:i+dim, j:j+dim])
            
            count += 1

In [6]:
# generate x
for f in os.listdir('./data/six_bands'):
    generate('./data/six_bands', f, './data/hed/', dim)